In [ ]:
# Import some necessary packages
import numpy as np
from src import utils, hiwa
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.manifold import Isomap
import time
import matplotlib.pyplot as plt

In [ ]:
# Load the datasets from a .mat file
T_te, T_tr, X_te, X_tr, Y_te, Y_tr = utils.load_data('data\\mihi_demo.mat')

# Apply initial dimensionality reduction
Y_te_3d = FactorAnalysis(n_components=3).fit_transform(utils.remove_const_cols(Y_te))
X_tr_3d = utils.map_X_3D(X_tr)
X_transform = np.linalg.pinv(X_tr_3d) @ X_tr

# Fit the model and transform the source dataset
t1 = time.time()
# This indicates to the model it should compute a low-d embedding with isomap, and that the parameter datasets will not be normalized
hwa = hiwa.HiWA(dim_red_method=Isomap(n_components=2), normalize=True)
# Fit the model, also passing in the correct rotation in Rgt since we can calculate it in this case (for computing error metrics)
# Note that we can provide the low-d mapping for either the source or target; the model will calculate whatever isn't provided using the technique
# indicated during initialization.
hwa.fit(Y_te_3d, T_te.squeeze(), X_tr_3d, T_tr.squeeze(), Y_transform=X_transform, Rgt=utils.LS_oracle(utils.map_X_3D(X_te), Y_te_3d))
# Transform the dataset with the learned rotation
Y_te_rec = hwa.transform(Y_te_3d)
t2 = time.time()
print(t2 - t1)

r2_hwa = utils.eval_R2(Y_te_rec, utils.map_X_3D(X_te))

In [ ]:
# Plot the results, plus some diagnostics
plt.figure(figsize=(20, 6))
plt.subplot(1, 5, 1)
plt.title('Target')
utils.color_data(X_tr, T_tr.squeeze())
plt.subplot(1, 5, 2)
plt.title('HiWA ($R^2 = $ %.4f)' %r2_hwa)
utils.color_data(Y_te_rec, T_te.squeeze())
ax = plt.subplot(1, 5, 3)
plt.plot(hwa.diagnostics['Rg_norm'])
ax.set_yscale('log')
plt.title('Residual Norm')
plt.xlabel('Iteration')
plt.subplot(1, 5, 4)
plt.imshow(hwa.P)
plt.xticks(np.arange(4))
plt.yticks(np.arange(4))
plt.title('Correspondence (Entropy$ = $ %.4f)' %(-np.sum(hwa.P * np.log(hwa.P))))
plt.subplot(1, 5, 5)
plt.title('Distance Matrix')
plt.imshow(hwa.diagnostics['C'])
c = plt.colorbar()

In [ ]:
# Load the datasets from a numpy archive
with np.load('data\\sleep_day1_60s.npz') as day1z:
    ft1 = day1z['firing_times_d1']
    l1 = day1z['labels_d1']
with np.load('data\\sleep_day2_60s.npz') as day2z:
    ft2 = day2z['firing_times_d2']
    l2 = day2z['labels_d2']

In [ ]:
# Apply initial dimensionality reduction
pca = PCA(n_components=3)
ft1_3d = pca.fit_transform(ft1)
ft2_3d = pca.fit_transform(ft2)

# Fit the model and transform the source dataset
t1 = time.time()
hwa = hiwa.HiWA(dim_red_method=PCA(n_components=2), normalize=True)
ft1_rec = hwa.fit_transform(ft1_3d, l1, ft2_3d, l2)
t2 = time.time()
print(t2 - t1)

In [ ]:
# Plot the results, plus some diagnostics
plt.figure(figsize=(20, 6))
plt.subplot(1, 5, 1)
plt.title('Target')
utils.color_data(ft2_3d, l2)
plt.subplot(1, 5, 2)
plt.title('HiWA')
utils.color_data(ft1_rec, l1)
ax = plt.subplot(1, 5, 3)
plt.plot(hwa.diagnostics['Rg_norm'])
ax.set_yscale('log')
plt.title('Residual Norm')
plt.xlabel('Iteration')
plt.subplot(1, 5, 4)
plt.imshow(hwa.P)
plt.xticks(np.arange(4))
plt.yticks(np.arange(4))
plt.title('Correspondence (Entropy$ = $ %.4f)' %(-np.sum(hwa.P * np.log(hwa.P))))
plt.subplot(1, 5, 5)
plt.title('Distance Matrix')
plt.imshow(hwa.diagnostics['C'])
c = plt.colorbar()

In [ ]:
# We're not limited to matching in 3D
pca = PCA(n_components=5)
ft1_5d = pca.fit_transform(ft1)
ft2_5d = pca.fit_transform(ft2)

# Fit the model and transform the source dataset
t1 = time.time()
hwa = hiwa.HiWA(dim_red_method=PCA(n_components=2), normalize=True)
ft2_rec = hwa.fit_transform(ft2_5d, l2, ft1_5d, l1)
t2 = time.time()
print(t2 - t1)

In [ ]:
# Plot the results, plus some diagnostics
plt.figure(figsize=(20, 6))
plt.subplot(1, 5, 1)
plt.title('Target')
utils.color_data(ft1_5d[:, 0:2], l1)
plt.subplot(1, 5, 2)
plt.title('HiWA')
utils.color_data(ft2_rec[:, 0:2], l2)
ax = plt.subplot(1, 5, 3)
plt.plot(hwa.diagnostics['Rg_norm'])
ax.set_yscale('log')
plt.title('Residual Norm')
plt.xlabel('Iteration')
plt.subplot(1, 5, 4)
plt.imshow(hwa.P)
plt.xticks(np.arange(4))
plt.yticks(np.arange(4))
plt.title('Correspondence (Entropy$ = $ %.4f)' %(-np.sum(hwa.P * np.log(hwa.P))))
plt.subplot(1, 5, 5)
plt.title('Distance Matrix')
plt.imshow(hwa.diagnostics['C'])
c = plt.colorbar()